In [ ]:
localuser = "argilla"
password = "1234"
apikey = "argilla.apikey"
url = "http://aicontroller:6900/"

In [ ]:
import argilla as rg
# get the docs here https://.hf.space/api/docs
# need to signin as owner with 12345678 to proceed the following.
# https://docs.argilla.io/en/latest/getting_started/installation/configurations/user_management.html
rg.init(
    api_url=url,
    api_key=apikey
)

In [ ]:
user = rg.User.create(
    username="new-user",
    first_name="New",
    last_name="User",
    password="12345678",
    role="annotator",
    workspaces=["argilla"]
)

In [ ]:
user

In [ ]:
rg.Workspace.create("localgilla")

In [ ]:
dataset = rg.FeedbackDataset.for_text_classification(
    labels=["sadness", "joy"],
    multi_label=False,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)

In [ ]:
dataset

In [ ]:
localgilla = "localgilla"

In [ ]:
dataset.push_to_argilla(name="ds-1", workspace=localgilla)

In [ ]:
records = [
    rg.FeedbackRecord(
        fields={
            "text": "I am so happy today",
        },
    ),
    rg.FeedbackRecord(
        fields={
            "text": "I feel sad today",
        },
    )
]
dataset.add_records(records)

In [ ]:
dataset.records

In [ ]:
dataset_hf = rg.FeedbackDataset.from_huggingface("argilla/emotion", split="train[1:101]")

In [ ]:
dataset_hf.push_to_argilla(name="emogilla", workspace=localgilla)

In [ ]:
from argilla.feedback import TrainingTask

task = TrainingTask.for_text_classification(
    text=dataset_hf.field_by_name('text'),
    label=dataset_hf.question_by_name("label")
)

In [ ]:
from argilla.feedback import ArgillaTrainer

trainer = ArgillaTrainer(
    dataset=dataset_hf,
    task=task,
    framework="transformers",
    train_size=0.8,
)

In [ ]:
trainer.get_trainer_model()

In [ ]:
trainer.get_trainer_tokenizer()

In [ ]:
trainer.get_model_kwargs()

In [ ]:
trainer.train(output_dir="train_arg_bert")

In [ ]:
### Uploading Data

# Three types of Record, depending on the tasks support TextClassificationRecord, 
# TokenClassificationRecord and Text2TextRecord

In [ ]:
rg.list_datasets()

In [ ]:
dataset = rg.FeedbackDataset.for_text_classification(
    labels=["good", "bad"],
    multi_label=False,
    use_markdown=True,
    guidelines=None,
    metadata_properties=None,
    vectors_settings=None,
)

In [ ]:
# Critical attrs for the record are text, annotation, prediction and metadata

textcat_rec = rg.TextClassificationRecord(
    text='Hello there. Its me',
    prediction=[("LABEL1", 0.8), ("LABEL2", 0.2)],
    annotation='LABEL1',
    multi_label=False
)

In [ ]:
tokencat_rec = rg.TokenClassificationRecord(
    text='Argilla is a super awesome library that speeds up annotation',
    tokens=["Argilla", "is", "a", "super", "awesome", "library", "that", "speeds",
            "up", "annotation"],
    prediction=[("Name", 0, 7), ("ADJ", 19, 26)],  # look at the indices, it matters
)

In [ ]:
text2text = rg.Text2TextRecord(
    text='Argilla is a super awesome library that speeds up annotation',
    prediction=["More we use it faster we understand it"],
)

In [ ]:
rg.set_workspace(localgilla)

In [ ]:
rg.log(textcat_rec, 'my_cat_ds')
rg.log(text2text, 'my_t2t_ds')
rg.log(tokencat_rec, 'my_tokenclass_ds')

In [ ]:
from datasets import load_dataset
ds = load_dataset("imdb", split='train').shuffle(seed=42).select(range(100))
ds.rename_column("label", "annotation")
df_rg = rg.read_datasets(ds,task="TextClassification")

In [ ]:
ds_in_arg = rg.log(df_rg, "imdb")

In [ ]:
ds_in_arg.schema()

In [ ]:
labels = ["pos", "neg"]
settings = rg.TextClassificationSettings(label_schema=labels)
rg.configure_dataset_settings(name="imdb", settings=settings)

In [ ]:
# tokenclassification task

from datasets import load_dataset

tokenDs = load_dataset("ag_news", split="train").shuffle(70).select(range(100))

In [ ]:
def metadata_dict(row):
    metadata = {}
    metadata['label'] = row["label"]
    row["metadata"] = metadata
    return row

dataset = tokenDs.map(metadata_dict, remove_columns=["label"])

In [ ]:
tokenDs

In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
def tokenizer(row):
    tokens = [token.text for token in nlp(row["text"])]
    return {"tokens": tokens}

In [ ]:
tokenDs = tokenDs.map(tokenizer)

In [ ]:
token_rg = rg.read_datasets(tokenDs, task="TokenClassification")

In [ ]:
rg.log(token_rg, 'ag_news')

In [ ]:
labels = ["PER", "LOC", "ORG", "MISC"]

settings = rg.TokenClassificationSettings(label_schema=labels)
rg.configure_dataset_settings(name="ag_news", settings=settings)

In [ ]:
t2tds = load_dataset("europa_ecdc_tm", "en2fr", split="train").shuffle(25).select(range(150))

In [ ]:
def extract(row):
    return {"text": row["translation"]["en"],
            "prediction": row["translation"]["fr"]}

t2tds = t2tds.map(extract, remove_columns=['translation'])

In [ ]:
t2trg = rg.read_datasets(t2tds, task="Text2Text")
rg.log(t2trg, "ecdc_en")

In [ ]:
import argilla as rg
# need to provide the reconrds
dataset = rg.FeedbackDataset(
    guidelines="Add some guidelines for the annotation team here.",
    fields=[
        rg.TextField(name="prompt", title="Human prompt"),
        rg.TextField(name="output", title="Generated output", use_markdown=True)
    ],
    questions =[
        rg.RatingQuestion(
            name="rating",
            title="Rate the quality of the response:",
            description="1 = very bad - 5= very good",
            required=True,
            values=[1,2,3,4,5]
        ),
        rg.TextQuestion(
            name="corrected-text",
            title="Provide a correction to the response:",
            required=False,
            use_markdown=True
        ),
        rg.LabelQuestion(
            name="relevant",
            title="Is the response relevant for the given prompt?",
            labels={"YES": "Yes", "NO": "No"}, # or ["YES","NO"]
            required=True,
            visible_labels=None
        ),
        rg.MultiLabelQuestion(
            name="content_class",
            title="Does the response include any of the following?",
            description="Select all that apply",
            labels={"hate": "Hate Speech" , "sexual": "Sexual content", "violent": "Violent content", "pii": "Personal information", "untruthful": "Untruthful info", "not_english": "Not English", "inappropriate": "Inappropriate content"}, # or ["hate", "sexual", "violent", "pii", "untruthful", "not_english", "inappropriate"]
            required=False,
            visible_labels=4
        ),
        rg.RankingQuestion(
            name="preference",
            title="Order replies based on your preference",
            description="1 = best, 3 = worst. Ties are allowed.",
            required=True,
            values={"reply-1": "Reply 1", "reply-2": "Reply 2", "reply-3": "Reply 3"} # or ["reply-1", "reply-2", "reply-3"]
        ),
        rg.MultiLabelQuestion(
            name="entities",
            title="Highlight the entities in the text:",
            labels={"PER": "Person", "ORG": "Organization", "EVE": "Event"},
            # or ["PER", "ORG", "EVE"],
            # field="text",
            required=True
        ),

    ]
)
# SpanQuestion errored out, so used MultiLabel question in its place
dataset.push_to_argilla(name="feedback_ds", workspace="argilla")

To query the rg.Dataset, you need to be able to write Lucene Query Language (LQL), which is native to Elastic Search and Open Search.

In [ ]:
# load the data record

record = rg.load(name="imdb", vector=("new_vector", [0, 43, 1985]))

In [ ]:
record = rg.TextClassificationRecord(
    text="Hello world, I am a vector record!",
    vectors= {"my_vector_name": [0, 42, 1984]}
)
rg.log(name="withvek", records=record)

In [ ]:
from argilla.labeling.text_classification import add_rules, delete_rules, Rule, update_rules

# Create
rule = Rule(query="positive impact", label="optimism")
add_rules(dataset="withvek", rules=[rule])

# Update
rule.label = "pessimism"
update_rules(dataset="withvek", rules=[rule])

# Delete
# delete_rules(dataset="withvek", rules=[rule])